In [13]:
# Import necessary libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.model_selection import train_test_split
import itertools
import random
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error


In [5]:
#df_All = pd.read_csv(r"C:\Users\saust\OneDrive - Sasol\1 Project rC4\Jupyter Notebooks\Report 10-20-23 No Fluff\df_All_Avg.csv")

df_All_1o2 = pd.read_csv('3 Feature Selection/contData_all_Avg - 1o2.csv')
df_All_1o2 = pd.read_csv('3 Feature Selection/contData_all_Avg - 2o2.csv')

#df_All = pd.read_csv(r"C:\Users\saust\OneDrive\Desktop\CodeSpace DLs\102423\Project rC4\3 Final Machine Butanol\df_CDCA6.csv")


In [6]:
# Concatenate (union) the dataframes
df_All = pd.concat([df_All_1o2, df_All_1o2], ignore_index=True)

print(df_All.head())

    425 %Al  Butanol   Decanol    Ethanol   Hexanol   Octanol  AYC55580  \
0  6.466835    8.950  3.271667  29.233325  5.862082  5.912500  9.708929   
1  6.479165    8.750  3.261668  29.233325  5.832917  5.870832  9.647859   
2  6.491500    8.525  3.310000  28.950000  5.767500  5.790000  9.592980   
3  6.503835    8.275  3.416667  28.383325  5.665832  5.670000  9.572995   
4  6.516165    8.025  3.523332  27.816675  5.564167  5.550000  9.586615   

    DI55102   DI55152   DI55580  ...    TI40050     TI52014     TI55013  \
0  1.016935  0.974092  0.991364  ...  74.705737  134.922750  204.003375   
1  1.017048  0.973535  0.991012  ...  74.042025  134.517750  203.384625   
2  1.017505  0.973474  0.990783  ...  73.626350  134.084875  202.217250   
3  1.017742  0.974004  0.990838  ...  73.203325  133.625625  201.412625   
4  1.017679  0.974445  0.991013  ...  72.900725  133.217250  201.149500   

      TI55014     TI55015     TI55016     TI55017     TI55021     TI55023  \
0  197.780625  193.19

In [7]:
# Set max columns to display
pd.set_option('display.max_columns', None)

In [8]:
# df_All = df_All[df_All['Date'] > '2022-06-15 00:00:00']

In [9]:
# List of columns to exclude to run XGboost feature selection
exclude_columns = ['Octanol', 'Hexanol',
       'Ethanol', 'Decanol',
       
       'TI52014', 'TI55013', 'TI55014', 'TI55015', 'TI55016', 'TI55017', 'TI55021', 'TI55023',
       'TC52015', 'FC52018', 'II52554', 'TI40050', 'VI52558B'

       # 'FC55102', 'FC55152', 'LC55557', 'LC55568', 'TC55555',

       # '425 SAO Al', 'FFC55553', 'LC52572', 'LC90366',

       # 'FC42428', 'LC55553',

       # 'FC55009'
                   ]

# Create a new DataFrame without the excluded columnsd
df_All = df_All.drop(columns=exclude_columns)

In [10]:
df_All.columns

Index(['425 %Al', 'Butanol', 'AYC55580', 'DI55102', 'DI55152', 'DI55580',
       'FC42428', 'FC55003', 'FC55009', 'FC55102', 'FC55152', 'FC55552',
       'FC55555', 'FC55569', 'FC55576', 'FFC55553', 'FFC55555', 'FYC55553',
       'LC52572', 'LC55553', 'LC55555', 'LC55557', 'LC55568', 'LC90366',
       'LC90368', 'PI55004', 'PI55020', 'PI55560', 'TC55552', 'TC55553',
       'TC55555', 'TC55566'],
      dtype='object')

In [11]:
# # Splitting into train and test
# X = df_All.drop('Butanol', axis=1)  # Assuming 'target' is your target column
# y = df_All['Butanol']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
def adjusted_r2(r_squared, num_data_points, num_predictors):
    return 1 - (1 - r_squared) * (num_data_points - 1) / (num_data_points - num_predictors - 1)

def iterate_feature_rotations(df_all, target_column, test_size=0.2, random_state=42):
    results = []
    columns = [col for col in df_all.columns if col != target_column]
    random.seed(random_state)  # for reproducibility

    for feature in columns:
        # Randomly order the remaining features
        remaining_features = [f for f in columns if f != feature]
        random.shuffle(remaining_features)

        # Create a new ordered list of features
        ordered_features = [feature] + remaining_features

        reordered_df = df_all[ordered_features + [target_column]]

        # Splitting into train and test for each permutation
        X = reordered_df.drop(target_column, axis=1)
        y = reordered_df[target_column]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

        # Create and fit the model using SFS
        linreg = LinearRegression()
        sfs = SFS(linreg, 
                  k_features='best', 
                  forward=True, 
                  floating=True, 
                  scoring='r2',
                  cv=5)
        sfs.fit(X_train, y_train)

        # Get the selected features
        selected_features = X_train.columns[list(sfs.k_feature_idx_)]

        # Build the final model using selected features and evaluate it
        final_model = linreg.fit(X_train[selected_features], y_train)
        y_pred = final_model.predict(X_test[selected_features])

        # Calculate performance metrics
        r2 = r2_score(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        rmse = mean_squared_error(y_test, y_pred, squared=False)
        mae = mean_absolute_error(y_test, y_pred)
        adj_r2 = adjusted_r2(r2, len(y_test), len(selected_features))

        # Store the result with the permutation order and performance metrics
        results.append((ordered_features, {'R2': r2, 'MSE': mse, 'RMSE': rmse, 'MAE': mae, 'Adjusted R2': adj_r2}))

    return results

results = iterate_feature_rotations(df_All, 'Butanol')

In [ ]:
# Get the selected feature names
selected_features = X_train.columns[list(sfs.k_feature_idx_)]
print("Selected Features:")
print(selected_features)

# Display the coefficients of the final model
print("\nModel Coefficients:")
for i, feature in enumerate(selected_features):
    print(f"{feature}: {final_model.coef_[i]}")

# Display the R-squared value for the training set
r_squared_train = final_model.score(X_train[selected_features], y_train)
print(f"\nR-squared on Training Set: {r_squared_train}")

# Optionally, if you evaluated the model on a test set
r_squared_test = final_model.score(X_test[selected_features], y_test)
print(f"R-squared on Test Set: {r_squared_test}")


Selected Features:
Index(['DI55102', 'DI55152', 'DI55580', 'FC42428', 'FC55003', 'FC55009',
       'FC55102', 'FC55152', 'FC55555', 'FC55569', 'FC55576', 'FFC55553',
       'FFC55555', 'FYC55553', 'LC52572', 'LC55553', 'LC55555', 'LC55557',
       'LC55568', 'LC90368', 'PI55004', 'PI55020', 'PI55560', 'TC55552',
       'TC55553', 'TC55555', 'TC55566'],
      dtype='object')

Model Coefficients:
DI55102: 19.090919963363447
DI55152: -12.641680219534832
DI55580: 41.934660411957395
FC42428: -0.0002428541787248711
FC55003: -0.0012700961037783443
FC55009: 0.0014358376554108392
FC55102: 0.0005645738519605126
FC55152: -0.0001493402703343454
FC55555: -0.003747565122712255
FC55569: -0.002027471723134855
FC55576: 0.00525126866687306
FFC55553: -125.3811420169266
FFC55555: 153.31584548368673
FYC55553: 0.002848473944735045
LC52572: 0.4162685690195336
LC55553: -0.18116019927770347
LC55555: 0.16421088111008822
LC55557: -0.8450680583951685
LC55568: 0.6262049400282906
LC90368: 0.01798279769016413
PI5500

: 

In [ ]:
# Selected Features:

# Inputs
# Index(['425 SAO Al', 'Butanol PPM', 'AYC55580', 'DI55102', 'DI55152',
#        'DI55580', 'FC42428', 'FC52018', 'FC55003', 'FC55009', 'FC55552',
#        'FC55555', 'FC55569', 'FC55576', 'FFC55553', 'FFC55555', 'FYC55553',
#        'II52554', 'LC52572', 'LC55553', 'LC55555', 'LC90366', 'LC90368',
#        'PI55004', 'PI55020', 'PI55560', 'TC52015', 'TC55552', 'TC55553',
#        'TC55566', 'TI40050', 'VI52558B'],
#       dtype='object')


# Outputs
# Index(['425 SAO Al', 'DI55152', 'FC55003', 'FFC55553', 'FFC55555', 'LC52572',
#        'LC55553', 'LC90366'],
#       dtype='object')

# Model Coefficients:
# 425 SAO Al: 30.01864005274383
# DI55152: 1078.681652177451
# FC55003: -0.08189077298629854
# FFC55553: 230.42968080660927
# FFC55555: -1435.533507758162
# LC52572: 5.674969686451699
# LC55553: -4.451261642303086
# LC90366: -0.9625483988093393

# R-squared on Training Set: 0.019547345671827254
# R-squared on Test Set: -0.1724414726438832

: 

In [ ]:
# Selected Features:

# Input
# Index(['425 SAO Al', 'Butanol PPM', 'AYC55580', 'DI55102', 'DI55152',
#        'DI55580', 'FC42428', 'FC52018', 'FC55003', 'FC55009', 'FC55102',
#        'FC55152', 'FC55552', 'FC55555', 'FC55569', 'FC55576', 'FFC55553',
#        'FFC55555', 'FYC55553', 'II52554', 'LC52572', 'LC55553', 'LC55555',
#        'LC55557', 'LC55568', 'LC90366', 'LC90368', 'PI55004', 'PI55020',
#        'PI55560', 'TC52015', 'TC55552', 'TC55553', 'TC55555', 'TC55566',
#        'TI40050', 'VI52558B'],
#       dtype='object')

# Output
# Index(['425 SAO Al', 'FC42428', 'FC55003', 'FC55102', 'FC55152', 'FFC55555',
#        'LC55553', 'LC55557', 'LC55568', 'LC90366', 'TC55555'],
#       dtype='object')

# Model Coefficients:
# 425 SAO Al: 25.17775577108802
# FC42428: -0.0053502077679223785
# FC55003: -0.08707793224835761
# FC55102: 0.009690441504333165
# FC55152: -0.00645465176065053
# FFC55555: -1225.499580989331
# LC55553: -3.790123484445094
# LC55557: 7.708962492130608
# LC55568: 22.260326246079796
# LC90366: -0.9876116671266065
# TC55555: 11.143149287396222

# R-squared on Training Set: 0.019324370488962517
# R-squared on Test Set: -0.16325654916601962

: 